

## Comparison of different genotypes between:

1. Marine's script used on original files
2. Stacks genotyping algorithm used on shortened files.
<br>
<br>
#### 10/3/2017

### Find matching loci between batch 1, corrected and batch 2, uncorrected

Note that when I run Marine's script, it gets rid of information about the locus_snp pairs. 

Batch 1 was processed with the random snp model, so there is only 1 possible snp per locus. The specific snp can be found in the `batch_1.genepop`. 

Batch 2 was processed to retain all snps, so there are more than one possible snp per locus. 

I need to find matching loci using the associated snp information, which can only be found in the `batch_1.genepop` and `batch_2.genepop` files. 

But I've already done this for the previous 2 notebooks. So I can just use the following files:

`batch_1_matched.CorrectedGenotypes_biallelic.txt`

`batch_2_matched.genepop`

And find the loci that match between the two. 

In [1]:
cd ../

/mnt/hgfs/Pacific cod/DataAnalysis/PCod-Compare-repo


In [9]:
MBfile = open("stacks_b1_wgenome/batch_1_matched.CorrectedGenotypes_biallelic.txt", "r")
stacksfile = open("stacks_b2_wgenome/batch_2_matched.genepop", "r")

header = stacksfile.readline()
loci_snp_list = stacksfile.readline().strip().split(",")
loci_list = [i.split("_")[0] for i in loci_snp_list]
stacksfile.close()


loci_list_MB = MBfile.readline().strip().split("\t")
MBfile.close()

matching = [i for i in loci_list_MB if i in loci_list]
print "There were ", len(matching), " matching loci."
not_matching = [i for i in loci_list_MB if i not in loci_list]
not_matching.append([i for i in loci_list if i not in loci_list_MB])
print "There were ", len(not_matching), " loci that did not match."


There were  6047  matching loci.
There were  891  loci that did not match.


In [36]:
# find indices of matched loci in Marine's file
MBfile = open("stacks_b1_wgenome/batch_1_matched.CorrectedGenotypes_biallelic.txt", "r")
MBout = open("stacks_b1_wgenome/batch_1_matched.CorrectedGenotypes_biallelic_matched_tostacks.txt", "w")

MBloci_list = MBfile.readline().strip().split()
index = 0
MBindices = []
new_first_line = ""
for i in MBloci_list:
    if i in matching:
        new_first_line += i + "\t"
        MBindices.append(index)
    index += 1
MBout.write(new_first_line + "\n")
    
# find indices of matched loci in the stacks file
stacksfile = open("stacks_b2_wgenome/batch_2_matched.genepop", "r")
stacksout = open("stacks_b2_wgenome/batch_2_matched_toMB.genepop", "w")
stacksout.write(stacksfile.readline())

loci_list = stacksfile.readline().strip().split(",")
index = 0
indices = []
new_first_line2 = ""
for i in loci_list:
    locus = i.split("_")[0]
    if locus in matching:
        new_first_line2 += locus + "\t"
        indices.append(index)
    index += 1
stacksout.write(new_first_line2 + "\n")

In [37]:
print MBindices[len(MBindices)-1]

6936


In [38]:
print len(MBindices)

6047


In [39]:
print indices[len(indices)-1]

11494


In [40]:
print len(indices)

6047


In [41]:
# write over genotypes for matching loci only 
for line in MBfile:
    if not line.startswith("pop"):
        new_line = ""
        linelist = line.strip().split("\t")
        new_line += linelist[0] + "\t"
        index = 0
        for i in range(1,len(linelist)):
            if i in MBindices and i != 6936:
                new_line += linelist[i] + "\t"
            elif i in MBindices and i == 6936:
                new_line += linelist[i] + "\n"
            index += 1
        MBout.write(new_line)
MBfile.close()
MBout.close()


for line in stacksfile:
    if not line.startswith("pop"):
        new_line = ""
        linelist = line.strip().split("\t")
        new_line += linelist[0] + "\t"
        index = 0
        for i in range(1,len(linelist)):
            if i in indices and i != 11494:
                new_line += linelist[i] + "\t"
            elif i in indices and i == 11494:
                new_line += linelist[i] + "\n"
            index += 1
        stacksout.write(new_line)
stacksfile.close()
stacksout.close()



#### 10/6/2017

In [1]:
cd ../

/mnt/hgfs/Pacific cod/DataAnalysis/PCod-Compare-repo


In [7]:
gen1 = open("stacks_b1_wgenome/batch_1_matched.CorrectedGenotypes_biallelic_matched_tostacks.txt", "r")
gen2 = open("stacks_b2_wgenome/batch_2_matched_toMB.genepop", "r")

#check to make sure that all needed locus_snp pairs are present in batch 2 genepop
loci1 = gen1.readline().strip().split("\t")
gen2.readline()
loci2 = gen2.readline().strip().split("\t")
for i in loci1:
    if i not in loci2: 
        print "oh no! You're missing locus ", i, " in your batch 2 file."
for i in loci2:
    if i not in loci1:
        print "oh no! You're missing locus ", i, " in your batch 1 file."

# make a dictionary where the key is the sample and the value is a list of genotypes
import collections

b2_genotype_dict = collections.OrderedDict()
for line in gen2:
    linelist = line.strip().split()
    if len(linelist) > 1:
        sample_name = linelist[0].strip("_subset,")
        b2_genotype_dict[sample_name] = linelist[1:]
gen2.close()      
        
b1_genotype_dict = collections.OrderedDict()
for line in gen1:
    linelist = line.strip().split()
    if len(linelist) > 1:
        sample_name = linelist[0].strip(",")
        if sample_name in b2_genotype_dict.keys():
            b1_genotype_dict[sample_name] = linelist[1:]
gen1.close()

In [10]:
print b2_genotype_dict.keys()

['PO010715_07_rep', 'PO010715_26', 'PO010715_27.1', 'PO020515_10.1', 'PO031715_04', 'PO031715_23', 'GE011215_08.1', 'GE011215_11', 'GE011215_30.1', 'GE012315_05.1', 'GE012315_10.1', 'GE012315_11_2', 'GEO012315_02', 'GEO012315_12', 'GEO012315_18', 'GEO012315_21', 'NA021015_02.1', 'NA021015_10.1', 'NA021015_25', 'YS_121316_18', 'YS_121316_21_2', 'YS_121316_28', 'YS_121316_29', 'JUK07_12', 'JUK07_13', 'JUK07_14', 'JUK07_15', 'JUK07_29.1', 'JUK07_32', 'JB121807_01', 'JB121807_05_2', 'JB121807_05', 'JB121807_12.1', 'JB121807_19.1', 'JB121807_23_2', 'JB121807_25', 'JB121807_33.1', 'JB121807_41.1', 'JB021108_25.1', 'JB021108_45', 'JB021108_46_rep.1', 'JB021108_48.1', 'BOR07_07.1', 'BOR07_10.1', 'BOR07_12.1', 'GEO020414_11', 'GEO020414_16', 'GEO020414_27', 'GEO020414_2', 'GEO020414_4']


In [11]:
print len(b2_genotype_dict['PO010715_07_rep'])

6047


In [12]:
print b1_genotype_dict.keys()

['PO010715_26', 'PO010715_27.1', 'PO020515_10.1', 'PO031715_04', 'PO031715_23', 'PO010715_07_rep', 'GE011215_08.1', 'GE011215_30.1', 'GE012315_05.1', 'GE012315_10.1', 'GEO012315_02', 'GEO012315_12', 'GEO012315_18', 'GEO012315_21', 'GE012315_11_2', 'GE011215_11', 'NA021015_02.1', 'NA021015_10.1', 'NA021015_25', 'YS_121316_18', 'YS_121316_28', 'YS_121316_29', 'YS_121316_21_2', 'JUK07_14', 'JUK07_15', 'JUK07_29.1', 'JUK07_32', 'JUK07_13', 'JUK07_12', 'JB121807_01', 'JB121807_05', 'JB121807_12.1', 'JB121807_19.1', 'JB121807_25', 'JB121807_33.1', 'JB121807_41.1', 'JB121807_05_2', 'JB121807_23_2', 'JB021108_25.1', 'JB021108_45', 'JB021108_46_rep.1', 'JB021108_48.1', 'BOR07_07.1', 'BOR07_10.1', 'BOR07_12.1', 'GEO020414_11', 'GEO020414_16', 'GEO020414_2', 'GEO020414_27', 'GEO020414_4']


In [13]:
print len(b1_genotype_dict['PO010715_26'])

6046


*WHY DO THE LENGTHS NOT MATCH?????*

I have bigger problems than that - the loci are listed in the same order. So I'm going to have to re-arrange the files..........


<br>
<br>
<br>

In [47]:
outfile = open("stacks_pipeline_analyses/analyses_AK_LowGenotypeRate/compare_KOR_genos_StacksDepth3_v_MBdepth10.txt", "w")

# write to an output file:
## --- (0) sample name
## --- (1) number of genotyped loci in batch 1
## --- (2) number of genotyped loci in batch 2

# calculate and write to output file:
## --- (3) number of same genotypes (includes missing genotypes)
## --- (4) number of same genotypes (does not include missing genotypes)

# calculate and write to output file:
## --- (5) number of different genotypes
## --- (5) number of different genotypes het in batch 1 --> hom in batch 2
## --- (6) number of different genotypes hom in batch 1 --> het in batch 2
## --- (7) number of different genotypes hom in batch 1 --> diff hom in batch 2
## --- (8) number of different genotypes het in batch 1 --> diff het in batch 2
## --- (9) number of different genotypes b/c batch 1 is missing, batch 2 is het
## --- (10) number of different genotypes b/c batch 1 is missing, batch 2 is hom
## --- (11) number of different genotypes b/c batch 2 is missing, batch 1 is het
## --- (12) number of different genotypes b/c batch 2 is missing, batch 2 is hom

outfile.write("sample\tgenotyped.b1\tgenotyped.b2\tsame.genos.wmissing\tsame.genos\tdiff.genos")
outfile.write("\tdiff.het_hom\tdiff.hom_het\tdiff.het_het\tdiff.hom_hom\tdiff.miss_het\tdiff.miss_hom\tdiff.het_miss\tdiff.hom_miss\n")


for sample in b2_genotype_dict.keys():
    outfile.write(sample + "\t")
    b2_genotypes = b2_genotype_dict[sample]
    b1_genotypes = b1_genotype_dict[sample]
    if len(b2_genotypes) != len(b1_genotypes):
        print "crap! you don't have the same number of genotypes in your batches."
        break
    # write to an output file: sample name, number genotyped per batch
    b1_genotyped = len([i for i in b1_genotypes if i != "0000"])
    b2_genotyped = len([i for i in b2_genotypes if i != "0000"])
    outfile.write(str(b1_genotyped) + "\t" + str(b2_genotyped) + "\t")
    # calculate and write to output file: matches and mismatches
    matched_wmissing = 0
    matched = 0
    different = 0
    diff_het_hom = 0
    diff_hom_het = 0
    diff_het_het = 0
    diff_hom_hom = 0
    diff_miss_hom = 0
    diff_miss_het = 0
    diff_het_miss = 0
    diff_hom_miss = 0
    for i in range(0, len(b1_genotypes)):
        # number of matching genotypes, with and without missing genotypes
        if b2_genotypes[i] == b1_genotypes[i]:
            if b2_genotypes[i] != "0000":
                matched += 1
                matched_wmissing += 1
            elif b2_genotypes[i] == "0000":
                matched_wmissing += 1
        # differing pairs, and the type of difference between them
        elif b2_genotypes[i] != b1_genotypes[i]:
            different += 1
            # if b1 is missing genotype
            if b1_genotypes[i] == "0000":
                if b2_genotypes[i][0:2] != b2_genotypes[i][2:]:
                    diff_miss_het += 1
                elif b2_genotypes[i][0:2] == b2_genotypes[i][2:]:
                    diff_miss_hom += 1
                else: 
                    print "your code is effed up! b1 missing"
            # if b2 is missing genotype
            elif b2_genotypes[i] == "0000":
                if b1_genotypes[i][0:2] != b1_genotypes[i][2:]:
                    diff_het_miss += 1
                elif b1_genotypes[i][0:2] == b1_genotypes[i][2:]:
                    diff_hom_miss += 1
                else: 
                    print "your code is effed up! b2 missing"
            # het --> het / hom
            elif b1_genotypes[i][0:2] != b1_genotypes[i][2:]:
                if b2_genotypes[i][0:2] == b2_genotypes[i][2:]:
                    diff_het_hom += 1
                elif b2_genotypes[i][0:2] != b2_genotypes[i][2:]:
                    diff_het_het += 1
            # hom --> het / hom
            elif b1_genotypes[i][0:2] == b1_genotypes[i][2:]:
                if b2_genotypes[i][0:2] != b2_genotypes[i][2:]:
                    diff_hom_het += 1
                elif b2_genotypes[i][0:2] == b2_genotypes[i][2:]:
                    diff_hom_hom += 1
            
    outfile.write(str(matched_wmissing) + "\t" + str(matched) + "\t" + str(different) + "\t")
    outfile.write(str(diff_het_hom) + "\t" + str(diff_hom_het) + "\t" + str(diff_het_het) + "\t" + str(diff_hom_hom) + "\t")
    outfile.write(str(diff_miss_het) + "\t" + str(diff_miss_hom) + "\t" + str(diff_het_miss) + "\t" + str(diff_hom_miss) + "\n")

outfile.close()

crap! you don't have the same number of genotypes in your batches.
